In [1]:
import pandas as pd
import plotly.plotly as py
from plotly.graph_objs import *
from plotly import tools
from datetime import datetime
def ConvertForPlots(dataframe):
    dataframe.fillna('',inplace=True)
    dataframe=dataframe.set_index('Location').T
    dataframe.reset_index(inplace=True)
    dataframe.rename(columns={dataframe.columns[0]:'Year'},inplace=True)
    dataframe.index.name='Index'
    return(dataframe)
def ConvertToEpoch(year):
    epoch = datetime(1970,1,1)
    i = datetime(year,1,1)
    delta_time = (i - epoch).total_seconds()
    return(delta_time*1000000000)
from sklearn import datasets, linear_model


In [85]:
sheepdata=ConvertForPlots(pd.read_csv('Sheep_Meat_Consumption.csv'))
poultrydata=ConvertForPlots(pd.read_csv('Poultry_Meat_Consumption.csv'))
pigdata=ConvertForPlots(pd.read_csv('Pig_Meat_Consumption.csv'))
beefdata=ConvertForPlots(pd.read_csv('Cow_Meat_Consumption.csv'))
countries=['China (People\'s Republic of)','India','United States']
colors = ['rgba(179,4,4,1)', 'rgba(0,148,27,1)','rgba(39,91,161,1)']
darkcolors = ['rgba(163,114,124,1)', 'rgba(118,163,114,1)','rgba(114,116,163,1)']
lightcolors = ['rgba(255,193,188,1)', 'rgba(61,234,9,1)','rgba(152,192,255,1)']

datasources=[sheepdata,poultrydata,pigdata,beefdata]
datatitles=['Mutton','Poultry','Pork','Beef']

In [3]:
fig=tools.make_subplots(rows=2,cols=2,shared_xaxes=False,shared_yaxes=False,vertical_spacing=.06)
gridx=[1,1,2,2]
gridy=[1,2,1,2]
for num in range(4):
    
    datatraces=[]
    for i in range(0,3):
        datatraces.append(
        Scatter(
            y=datasources[num][countries[i]],
            x=datasources[num]['Year'],
            line=Line(
                color=colors[i], width=4),
               
            name=countries[i],
            hoverinfo="name+x+y",
            showlegend=(num==0)
            )
    
        )
#fig=Figure(data=data,layout=layout)
    fig.append_trace(datatraces[0],gridx[num],gridy[num])
    fig.append_trace(datatraces[1],gridx[num],gridy[num])
    fig.append_trace(datatraces[2],gridx[num],gridy[num])
fig['layout'].update(height=1000,legend=Legend(y=-.1,x=.5,yanchor='middle',xanchor='center'),title='Meat Consumption in China, India, and USA',plot_bgcolor= 'rgb(215, 229, 238)',font=Font(family='Droid Sans') )
fig['layout']['yaxis1'].update(title=datatitles[0]+' (kg consumed per capita)',fixedrange=True, zerolinecolor="rgb(255, 255, 255)" )
fig['layout']['yaxis2'].update(title=datatitles[1]+' (kg consumed per capita)', fixedrange=True,zerolinecolor="rgb(255, 255, 255)" )
fig['layout']['yaxis3'].update(title=datatitles[2]+' (kg consumed per capita)', fixedrange=True,zerolinecolor="rgb(255, 255, 255)" )
fig['layout']['yaxis4'].update(title=datatitles[3]+' (kg consumed per capita)', fixedrange=True,zerolinecolor="rgb(255, 255, 255)" )
fig['layout']['xaxis1'].update( type="linear", range=[1990, 2015 ], autorange=False, fixedrange=True,dtick=1,gridwidth=3,showgrid=False)
fig['layout']['xaxis2'].update( type="linear", range=[1990, 2015 ], autorange=False, fixedrange=True,dtick=1,gridwidth=3,showgrid=False)
fig['layout']['xaxis3'].update( type="linear", range=[1990, 2015 ], autorange=False, fixedrange=True,dtick=1,gridwidth=3,showgrid=False)
fig['layout']['xaxis4'].update( type="linear", range=[1990, 2015 ], autorange=False, fixedrange=True,dtick=1,gridwidth=3,showgrid=False)

py.iplot(fig, filename = 'meat_consumption_china+india+usa')


This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



#Arable Land
Now we're looking at Arable Land in the form of hectares per person. Because the data from the World Bank only extends to 2012, I applied a quick linear regression (predicted values are in the last three years to 2015). 

In [19]:
arable_land=ConvertForPlots(pd.read_csv('arable_land.csv'))
fertilizer_utilization=ConvertForPlots(pd.read_csv('fertilizer_utilization.csv'))

In [27]:
forecast_years=range(2012,2016)
landtraces=[]
for i in range(0,3):
    linear_regression=[]
    landtraces.append(
    Scatter(
        y=arable_land[countries[i]],
        x=arable_land['Year'],
        line=Line(
                color=colors[i], width=4),
            name=countries[i],
            hoverinfo="x+y",
            showlegend=True
            )
    
        )
    regr = linear_model.LinearRegression()
    regr.fit(pd.to_datetime(arable_land['Year'][30:52]).astype(int).reshape(22,1),arable_land[countries[i]][30:52].convert_objects(convert_numeric=True).reshape(22,1))
    linear_regression.append(arable_land[countries[i]][52])
    prediction=regr.predict(pd.DataFrame(map(ConvertToEpoch,forecast_years)))
    linear_regression.extend(item[0] for item in prediction)
    landtraces.append(
    Scatter(
        x=forecast_years,
        y=linear_regression,
        line=Line(
            color=colors[i],
            dash='dot'),
        mode='lines',
        name=countries[i]+' (predicted)',
            
        showlegend=False
        ))
    
layout=Layout(height=500,legend=Legend(y=-.17,x=.5,yanchor='middle',xanchor='center'),title='Arable Land in China, India, and USA ',plot_bgcolor= 'rgb(215, 229, 238)',font=Font(family='Droid Sans'),xaxis=XAxis(range=[1990,2015],dtick=1),yaxis=YAxis(title='Hectares per person'))
# landtraces.append(
# Scatter(
#     x=years,
#     y=regr.predict(pd.DataFrame(conv))
    
#     ))
fig=Figure(data=Data(landtraces),layout=layout)

py.iplot(fig, filename = 'arab1le_land_china+india+usa')


In [88]:
forecast_years=range(2012,2016)
prediction=[]
landtraces=[]
for i in range(0,2):
    arable_regression=[]
    fertilizer_regression=[]
    landtraces.append(
    Scatter(
        y=fertilizer_utilization[countries[i]],
        x=fertilizer_utilization['Year'],
        line=Line(
                color=lightcolors[i], width=5,
                dash='dot'),
            name=countries[i]+' Fertilizer Use',
            hoverinfo="x+y",
            showlegend=True,
            legendgroup='Fertilizer',
            
        yaxis='y2',)
    
        )
    landtraces.append(
    Scatter(
        y=arable_land[countries[i]],
        x=arable_land['Year'],
        line=Line(
                color=colors[i], width=5),
            name=countries[i]+' Arable Land',
            hoverinfo="x+y",
            showlegend=True,
            legendgroup='ArableLand'
            )
    
        )
    arable_regr = linear_model.LinearRegression()
    arable_regr.fit(pd.to_datetime(arable_land['Year'][30:52]).astype(int).reshape(22,1),arable_land[countries[i]][30:52].convert_objects(convert_numeric=True).reshape(22,1))
    arable_regression.append(arable_land[countries[i]][52])
    prediction=arable_regr.predict(pd.DataFrame(map(ConvertToEpoch,forecast_years)))
    arable_regression.extend(item[0] for item in prediction)
    fertilizer_regr = linear_model.LinearRegression()
    fertilizer_regr.fit(pd.to_datetime(fertilizer_utilization['Year'][42:52]).astype(int).reshape(10,1),fertilizer_utilization[countries[i]][42:52].convert_objects(convert_numeric=True).reshape(10,1))
    fertilizer_regression.append(fertilizer_utilization[countries[i]][52])
    prediction2=fertilizer_regr.predict(pd.DataFrame(map(ConvertToEpoch,forecast_years)))
    fertilizer_regression.extend(item[0] for item in prediction2)

    landtraces.append(
    Scatter(
        x=forecast_years,
        y=arable_regression,
        line=Line(
            color=colors[i],dash='dash'),
        mode='lines',
        name=countries[i]+' Arable Land (predicted)',
        hoverinfo="x+y",
            
        showlegend=False
        ))
    landtraces.append(
    Scatter(
        x=forecast_years,
        y=fertilizer_regression,
        hoverinfo="x+y",
        line=Line(
            color=lightcolors[i],dash='dash'),
        mode='lines',
        name=countries[i]+' Fertilizer Use (predicted)',
        yaxis='y2',
        showlegend=False
        ))
    
layout=Layout(height=700,legend=Legend(y=-.2,x=.5,yanchor='middle',xanchor='center'),title='Fertilizer Use & Arable Land',plot_bgcolor= 'rgba(215, 229, 238, 0.23) ',font=Font(family='Droid Sans'),xaxis=XAxis(range=[2002,2015],dtick=1),yaxis=YAxis(title='Arable Land (hectares per person)',side='right'),yaxis2=YAxis(overlaying='y',side='left',title='Fertilizer Use (kg per hectare)'))
# landtraces.append(
# Scatter(
#     x=years,
#     y=regr.predict(pd.DataFrame(conv))
    
#     ))
fig=Figure(data=Data(landtraces),layout=layout)

py.iplot(fig, filename = 'Fertilizer_Use')
